In [66]:
import math
import time
import numpy as np
import json
import glob
import warnings
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)


import datetime



# Import der Daten

In [67]:
df = pd.read_pickle('dfipynb.pickle')
dftmp=df

# Datumszeile erstellen

In [68]:
def anpassung_zeit(df):
    df = df[df['Querschnitt'].notna()]
    df = df[df['Datum'].notna()]
    df.HHMM = df.HHMM / 100
    df['HHMM']=df['HHMM'].replace(24.0,23.59)
    df.head(100)
    df['Datum und Zeit'] = df["Datum"].astype(str).str[:10]+ ' ' +df.HHMM.astype(str)
    df['Datumszeit'] = pd.to_datetime(df['Datum und Zeit'], format='%Y-%m-%d %H.%M', errors= "coerce")
    return df
df=anpassung_zeit(df)

In [69]:
df

,Zst,Datum,HHMM,Richtung,-,Klasse,CS 1,CS 2,CS 3,CS 4,CS 5,CS 6,CS 7,CS 8,CS 9,CS10,Day Helper,Month Helper,Querschnitt,Wochenende/Feirtag,Datum und Zeit,Datumszeit
1,5401,2022-01-01 00:00:00,1.00,1.0,0.0,2.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,20.0,x,2022-01-01 1.0,2022-01-01 01:00:00
3,5401,2022-01-01 00:00:00,1.00,2.0,0.0,2.0,1.0,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,23.0,x,2022-01-01 1.0,2022-01-01 01:00:00
5,5401,2022-01-01 00:00:00,2.00,1.0,0.0,2.0,1.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,32.0,x,2022-01-01 2.0,2022-01-01 02:00:00
7,5401,2022-01-01 00:00:00,2.00,2.0,0.0,2.0,1.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,38.0,x,2022-01-01 2.0,2022-01-01 02:00:00
9,5401,2022-01-01 00:00:00,3.00,1.0,0.0,2.0,1.0,1.0,23.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,26.0,x,2022-01-01 3.0,2022-01-01 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3342363,9005.0,2021-12-31 00:00:00,22.00,2.0,0.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,4.0,NaN,2021-12-31 22.0,2021-12-31 22:00:00
3342365,9005.0,2021-12-31 00:00:00,23.00,1.0,0.0,2.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,9.0,NaN,2021-12-31 23.0,2021-12-31 23:00:00
3342367,9005.0,2021-12-31 00:00:00,23.00,2.0,0.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,4.0,NaN,2021-12-31 23.0,2021-12-31 23:00:00
3342369,9005.0,2021-12-31 00:00:00,23.59,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,3.0,NaN,2021-12-31 23.59,2021-12-31 23:59:00


In [70]:
tmp = df
index = pd.DatetimeIndex(tmp['Datumszeit'])
tmp = tmp.iloc[index.indexer_between_time('06:30', '22:30')]
tmp = tmp[tmp['Datumszeit'].dt.year == 2022]
# group by measurepoint and date, then apply aggregations
result = tmp.groupby(['Zst', 'Datumszeit']).agg({'Querschnitt': ['sum']}).reset_index()
result["Datum"] =  result["Datumszeit"].dt.date
result.columns = [''.join(col).strip() for col in result.columns.values]
result = result.groupby(['Zst', 'Datum']).agg({'Querschnittsum': ['sum','min','max']}).reset_index()
result.columns = ['_'.join(col).strip() for col in result.columns.values]
result['diff'] = result['Querschnittsum_max']-result['Querschnittsum_min']
result['avg/h'] = result['Querschnittsum_sum']/17

In [71]:
result["Zst_"]=result["Zst_"].replace([5401, 6011, 6202, 6015, 6003, 6009, 6025, 6002, 6016, 6004, 6010, 6023, 6021, 6020, 6024,
6018, 6022, 6013, 6014, 6017, 6026, 6303, 6019, 6707, 6706, 6704, 6705, 6708, 6701, 6703, 7208, 7202, 7201, 7408, 7501, 7403, 8405,
8404, 8402, 8401, 8703, 8702, 9004, 9006, 9005, 9301, 9101, 7701, 6501],
['Bahnhof_Flüelen','Allmendstrasse', 'Hess', 'Ara Altdorf', 'JB Bau', 'Spital', 'EWA Altdorf', 'Schlosserei Trögli', 'Bifang',
 'Schulhaus Hagen', 'Staatsarchiv', 'Staatsarchiv2', 'Dätwyler', 'Gisler Druck', 'St. Karl',
 'Amavita Apotheke', 'Bahnhof Altdorf Bauernhof', 'Steinmatt Merck', 'Ehem. Kebab Hüsli', 'Kant. Verwaltung', 'Kollegium',
  'Strassenverkehrsamt', 'Ruberst', 'Grund Dorfstrasse', 'Grund Gotthardstrasse', 'Gandrütti', 'Adlergarten', 'Sisag',
   'Elektro Nauer', 'Jumbo', 'Breiteli', 'Birtschen', 'Erstfeld-Silenen', 'Silenen-Amsteg', 'Galleri Breitlaui',
   'Amsteg', 'Husen', 'Werkhalle Wassen', 'Nördlich einfahrt A2 Wassen Periodisch', 'Bahnhof Wassen', 'Teufelsstein Periodisch',
   'Südlich Keisel Göschenen', 'Andermatt Dorf', 'Nätschen', "Tristelboden", 'Feldbäckerei VBS', 'Tiefenbach', "Seelisberg",'Urnerboden'])

In [72]:
result

,Zst_,Datum_,Querschnittsum_sum,Querschnittsum_min,Querschnittsum_max,diff,avg/h
0,Bahnhof_Flüelen,2022-01-01,2731.0,46.0,353.0,307.0,160.647059
1,Bahnhof_Flüelen,2022-01-02,3084.0,46.0,348.0,302.0,181.411765
2,Bahnhof_Flüelen,2022-01-03,3771.0,89.0,390.0,301.0,221.823529
3,Bahnhof_Flüelen,2022-01-04,3876.0,91.0,347.0,256.0,228.000000
4,Bahnhof_Flüelen,2022-01-05,3830.0,89.0,412.0,323.0,225.294118
...,...,...,...,...,...,...,...
12683,Südlich Keisel Göschenen,2022-12-27,722.0,17.0,81.0,64.0,42.470588
12684,Südlich Keisel Göschenen,2022-12-28,637.0,12.0,64.0,52.0,37.470588
12685,Südlich Keisel Göschenen,2022-12-29,636.0,10.0,67.0,57.0,37.411765
12686,Südlich Keisel Göschenen,2022-12-30,742.0,8.0,80.0,72.0,43.647059


In [73]:
import plotly.express as px
fig = px.histogram(result, x='Zst_', y='avg/h', height=750)

fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.update_xaxes(type='category')

fig.show()

In [74]:
import plotly.express as px


fig = px.histogram(result, x='Zst_', y='diff', height=600)
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})

fig.update_xaxes(type='category')

fig.show()

In [75]:
import plotly.express as px
result['rel'] = result['diff']/result['avg/h']

fig = px.histogram(result, x='Zst_', y='rel', height=700)

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.update_xaxes(type='category')

fig.show()

In [76]:
fig = px.line(result, x="Datum_", y="Querschnittsum_sum", color='Zst_', height=700)

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=7,
                     label="W",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [77]:
fig = px.line(result, x="Datum_", y="avg/h", color='Zst_', height=700)

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=7,
                     label="W",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()
fig.write_html("PlotZählstellen")

In [78]:
fig = px.line(result, x="Datum_", y="Querschnittsum_sum", color='Zst_', height=500)

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=7,
                     label="W",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()
fig.write_html("PlotZählstellen")

In [79]:
result

,Zst_,Datum_,Querschnittsum_sum,Querschnittsum_min,Querschnittsum_max,diff,avg/h,rel
0,Bahnhof_Flüelen,2022-01-01,2731.0,46.0,353.0,307.0,160.647059,1.911022
1,Bahnhof_Flüelen,2022-01-02,3084.0,46.0,348.0,302.0,181.411765,1.664721
2,Bahnhof_Flüelen,2022-01-03,3771.0,89.0,390.0,301.0,221.823529,1.356935
3,Bahnhof_Flüelen,2022-01-04,3876.0,91.0,347.0,256.0,228.000000,1.122807
4,Bahnhof_Flüelen,2022-01-05,3830.0,89.0,412.0,323.0,225.294118,1.433681
...,...,...,...,...,...,...,...,...
12683,Südlich Keisel Göschenen,2022-12-27,722.0,17.0,81.0,64.0,42.470588,1.506925
12684,Südlich Keisel Göschenen,2022-12-28,637.0,12.0,64.0,52.0,37.470588,1.387755
12685,Südlich Keisel Göschenen,2022-12-29,636.0,10.0,67.0,57.0,37.411765,1.523585
12686,Südlich Keisel Göschenen,2022-12-30,742.0,8.0,80.0,72.0,43.647059,1.649596
